# Vision-based Lung Function Reading

#### Current Pipeline: 
* Measures displacement of chest in real-time
* Visualizes 3D rendering of chest in real-time
* Visualizes chest displacement in real-time using graph
* Skeleton tracking for position and movement detection (neck, shoulder, side-to-side, rocking, and legs)
* User keyboard control for baseline parameters of positioning, initating lung function measures, laser, and volume/depth
* Computes lung function parameters (in another cell & creates a weird display error when running script again after)

#### TODOS / For Pitch Demo:
* (x) Make movement labels bigger and clearer 
* (x) Add 'Chest' label 
* (x) Maybe make box even bigger 
* (x) Apply distance metric along with chest ROI 
* (x) Add lung parameter box that automatically fills in
* Save RGB video 

#### TODOS / Next Steps:
* Better understanding frames per second of camera and implementing that in the code 
* Making lung params more robust for each of the values 
* Feature to compute Chest width, Sitting height, and waist measurements 
* Incorporate optional I2L-MeshNet for full body mesh measurement  
* Display translated graphs and lung function params in current GUI interface 
* Bring code to cloud & GITHUB & Make Technical Documentation 

In [1]:
print('a')

a


### Package Imports

In [2]:
#!/usr/bin/env python3
import matplotlib.animation as animation

import joblib
from scipy.signal import find_peaks

import sys
sys.path.append('C:\Program Files\Cubemos\SkeletonTracking\samples\python')
sys.path.append('.\opencv-plot-master')

import matplotlib.pyplot as plt
import os
import numpy as np

from collections import namedtuple
import util as cm
import cv2
import time
import pyrealsense2 as rs
import math
import numpy as np 
from skeletontracker import skeletontracker
import cv2

from skimage.measure import label, regionprops, regionprops_table
from skimage.filters import threshold_otsu
from skimage.morphology import closing, square

import random

from scipy.signal import savgol_filter, lfilter
from scipy.signal import lfilter

from pyntcloud import PyntCloud

import pandas as pd

%matplotlib qt5

### Helper Functions

In [3]:
# =====================================================
# SKELETON TRACKING FUNCTIONS
# =====================================================
def render_ids_3d(render_image, skeletons_2d, depth_map, depth_intrinsic, joint_confidence):
    
    thickness = 1
    text_color = (255, 255, 255)
    rows, cols, channel = render_image.shape[:3]
    distance_kernel_size = 5
    
    # calculate 3D keypoints and display them
    for skeleton_index in range(len(skeletons_2d)):
        skeleton_2D = skeletons_2d[skeleton_index]
        joints_2D = skeleton_2D.joints
        did_once = False
        for joint_index in range(len(joints_2D)):
            if did_once == False:
                cv2.putText(
                    render_image,
                    "id: " + str(skeleton_2D.id),
                    (int(joints_2D[joint_index].x), int(joints_2D[joint_index].y - 30)),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.55,
                    text_color,
                    thickness,
                )
            did_once = True
            # check if the joint was detected and has valid coordinate
            if skeleton_2D.confidences[joint_index] > joint_confidence:
                distance_in_kernel = []
                low_bound_x = max(
                    0,
                    int(
                        joints_2D[joint_index].x - math.floor(distance_kernel_size / 2)
                    ),
                )
                upper_bound_x = min(
                    cols - 1,
                    int(joints_2D[joint_index].x + math.ceil(distance_kernel_size / 2)),
                )
                low_bound_y = max(
                    0,
                    int(
                        joints_2D[joint_index].y - math.floor(distance_kernel_size / 2)
                    ),
                )
                upper_bound_y = min(
                    rows - 1,
                    int(joints_2D[joint_index].y + math.ceil(distance_kernel_size / 2)),
                )
                for x in range(low_bound_x, upper_bound_x):
                    for y in range(low_bound_y, upper_bound_y):
                        distance_in_kernel.append(depth_map.get_distance(x, y))
                median_distance = np.percentile(np.array(distance_in_kernel), 50)
                depth_pixel = [
                    int(joints_2D[joint_index].x),
                    int(joints_2D[joint_index].y),
                ]
                if median_distance >= 0.3:
                    point_3d = rs.rs2_deproject_pixel_to_point(
                        depth_intrinsic, depth_pixel, median_distance
                    )
                    point_3d = np.round([float(i) for i in point_3d], 3)
                    point_str = [str(x) for x in point_3d]
                    
#                     cv2.putText(
#                         render_image,
#                         str(point_3d),
#                         (int(joints_2D[joint_index].x), int(joints_2D[joint_index].y)),
#                         cv2.FONT_HERSHEY_DUPLEX,
#                         0.4,
#                         text_color,
#                         thickness,
#                     )

# =====================================================
# MOVEMENT & POSITION TRACKING FUNCTIONS
# =====================================================
                    
def compute_orientation(img, display=False):
    
    # apply threshold
    thresh = threshold_otsu(img)
    bw = closing(img > thresh, square(3))

    label_img = label(bw)
    props = regionprops(label_img)

    # largest img
    largest_index = np.argmax([p.area for p in props])
    prop = props[largest_index]
    
    if display: 

        plt.figure()
        plt.imshow(prop.image)

        x0 = prop['Centroid'][1]
        y0 = prop['Centroid'][0]
        x2 = x0 - math.sin(prop['Orientation']) * 0.9 * prop['MinorAxisLength']
        y2 = y0 - math.cos(prop['Orientation']) * 0.9 * prop['MinorAxisLength']

        plt.plot((x0, x2), (y0, y2), '-r', linewidth=2.5)
        plt.plot(x0, y0, '.g', markersize=15)

        minr, minc, maxr, maxc = prop['BoundingBox']
        bx = (minc, maxc, maxc, minc, minc)
        by = (minr, minr, maxr, maxr, minr)
        
        plt.plot(bx, by, '-b', linewidth=2.5)
        plt.show()
        
    return prop.orientation

def detect_legs(img, skeleton): 
    
    x_rk, y_rk = int(skeleton.joints[9][0]), int(skeleton.joints[9][1])
    x_ra, y_ra = int(skeleton.joints[10][0]), int(skeleton.joints[10][1])
    x_lk, y_lk = int(skeleton.joints[12][0]), int(skeleton.joints[12][1])
    x_la, y_la = int(skeleton.joints[13][0]), int(skeleton.joints[13][1])
    
    # zero out each leg
    img_left_leg = np.zeros(img.shape,np.uint8)
    img_left_leg[y_lk:y_la, x_la-30:x_la+30] = img[y_lk:y_la, x_la-30:x_la+30]
    img_right_leg = np.zeros(img.shape, np.uint8)
    img_right_leg[y_rk:y_ra, y_ra-30:y_ra+30] = img[y_rk:y_ra, x_ra-30:x_ra+30]

    # threshold each leg 
    gray_left = cv2.cvtColor(img_left_leg, cv2.COLOR_BGR2GRAY)
    ret_left, thresh_left = cv2.threshold(gray_left, 0, 255, cv2.THRESH_OTSU)
    gray_right = cv2.cvtColor(img_right_leg, cv2.COLOR_BGR2GRAY)
    ret_right, thresh_right = cv2.threshold(gray_right, 0, 255, cv2.THRESH_OTSU)

    # find contours for each leg 
    x_l, y_l, w_l, h_l = cv2.boundingRect(thresh_left)
    x_r, y_r, w_r, h_r = cv2.boundingRect(thresh_right)

#     final_img = cv2.rectangle(img, (x_l, y_l), (x_l + w_l, y_l + h_l), (36,255,12), 2)
#     final_img = cv2.rectangle(final_img, (x_r, y_r), (x_r + w_r, y_r + h_r), (36,255,12), 2)

    # display
    #cv2.putText(final_img, 'left leg', (x_l, y_l-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
    #cv2.putText(final_img, 'right leg', (x_r, y_r-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

    cv2.imshow('image', final_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
# distance formula
def distance(x1, y1, x2,y2):
    dist = math.sqrt((math.fabs(x2-x1))**2 + ((math.fabs(y2-y1)))**2)
    return dist

def compute_angle(x1, y1, x2, y2):
    
    # quantifies the hypotenuse of the triangle
    hypotenuse =  distance(x1, y1, x2, y2)
    
    #quantifies the horizontal of the triangle
    horizontal = distance(x1, y1, x2, y1)
    
    # makes the third-line of the triangle
    thirdline = distance(x2, y2, x2, y1)
    
    #calculates the angle using trigonometry
    angle = np.arcsin((thirdline / hypotenuse)) * 180 / math.pi
    
    return angle
    
def detect_leg_pos_and_angle(img, skeleton):
    
    x_rk, y_rk = int(skeleton.joints[9][0]), int(skeleton.joints[9][1])
    x_ra, y_ra = int(skeleton.joints[10][0]), int(skeleton.joints[10][1])
    
    x_lk, y_lk = int(skeleton.joints[12][0]), int(skeleton.joints[12][1])
    x_la, y_la = int(skeleton.joints[13][0]), int(skeleton.joints[13][1])
    
    left_angle = compute_angle(x_lk, y_lk, x_la, y_la)
    right_angle = compute_angle(x_rk, y_rk, x_ra, y_ra)
    
    #(255,12,36), (36,255,12)
    if (left_angle <= 80 or left_angle >= 100) or (right_angle <= 80 or right_angle >= 100): 
        cv2.putText(img, 'Leg Position: Bad', (20, 140), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)
        return 0
    else: 
        cv2.putText(img, 'Leg Position: Good', (20, 140), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2)
        return 1
        
def detect_leg_pos_old(img, skeleton, display=False): 
    
    x_rk, y_rk = int(skeleton.joints[9][0]), int(skeleton.joints[9][1])
    x_ra, y_ra = int(skeleton.joints[10][0]), int(skeleton.joints[10][1])
    x_lk, y_lk = int(skeleton.joints[12][0]), int(skeleton.joints[12][1])
    x_la, y_la = int(skeleton.joints[13][0]), int(skeleton.joints[13][1])
        
    # zero out each leg
    img_left_leg = np.zeros(img.shape,np.uint8)
    img_left_leg[y_lk:y_la-15, x_la-70:x_la+70] = img[y_lk:y_la-15, x_la-70:x_la+70]
    img_right_leg = np.zeros(img.shape, np.uint8)
    img_right_leg[y_rk:y_ra-15, x_ra-70:x_ra+70] = img[y_rk:y_ra-15, x_ra-70:x_ra+70]

    # threshold each leg 
    gray_left = cv2.cvtColor(img_left_leg, cv2.COLOR_BGR2GRAY)
    ret_left, thresh_left = cv2.threshold(gray_left, 0, 255, cv2.THRESH_OTSU)
    gray_right = cv2.cvtColor(img_right_leg, cv2.COLOR_BGR2GRAY)
    ret_right, thresh_right = cv2.threshold(gray_right, 0, 255, cv2.THRESH_OTSU)

    # obtain contour
    cntrs_left = cv2.findContours(thresh_left, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cntrs_left = cntrs_left[0] if len(cntrs_left) == 2 else cntrs_left[1]
    cntrs_left = max(cntrs_left, key=cv2.contourArea)
    cntrs_right = cv2.findContours(thresh_right, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cntrs_right = cntrs_right[0] if len(cntrs_right) == 2 else cntrs_right[1]
    cntrs_right = max(cntrs_right, key=cv2.contourArea)

    # get bounding box for legs
    x_l, y_l, w_l, h_l = cv2.boundingRect(cntrs_left)
    x_r, y_r, w_r, h_r = cv2.boundingRect(cntrs_right)

    # fit ellipsoid to find orientation 
    ellipse_left, ellipse_right = cv2.fitEllipse(cntrs_left), cv2.fitEllipse(cntrs_right)

    _, angle_l = get_angle_orienation(img, ellipse_left)
    _, angle_r = get_angle_orienation(img, ellipse_right)
    
    print('ANGLES: ', angle_l, angle_r)

    # display 
    if angle_l >= 40 and angle_l <= 120: 
        color_l = (36,255,12)
        status_l = True
    else: 
        status_l = False

    if angle_r >= 50 and angle_r <= 100:
        color_r = (36,255,12)
        status_r = True
    else: 
        status_r = False
        
    cv2.putText(img, str(round(angle_l, 1)), (x_l, y_l-20), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,12,36), 2)
    cv2.putText(img, str(round(angle_r, 1)), (x_r, y_r-20), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,12,36), 2)
    
    if not status_r or not status_l: 
        cv2.putText(img, 'Leg Position: Bad', (20, 140), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 3)
        
        #cv2.putText(img, str(round(angle_l, 1)), (x_l, y_l-20), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,12,36), 2)
        cv2.rectangle(img, (x_l, y_l), (x_l + w_l, y_l + h_l), (255,12,36), 2)

        #cv2.putText(img, str(round(angle_r, 1)), (x_r, y_r-20), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,12,36), 2)
        cv2.rectangle(img, (x_r, y_r), (x_r + w_r, y_r + h_r), (255,12,36), 2)
    else: 
        cv2.putText(img, 'Leg Position: Good', (20, 140), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 3)
    
    if display: 
        cv2.imshow('image', img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
    return (status_r and status_l)
        
def get_angle_orienation(img, ellipse, draw_line=False): 
    
    result = None
    (xc, yc), (d1, d2), angle = ellipse
    
    # draw orienation line
    rmajor = max(d1, d2) / 2
    if angle > 90:
        angle = angle - 90
    else:
        angle = angle + 90

    xtop = xc + math.cos(math.radians(angle))*rmajor
    ytop = yc + math.sin(math.radians(angle))*rmajor
    xbot = xc + math.cos(math.radians(angle+180))*rmajor
    ybot = yc + math.sin(math.radians(angle+180))*rmajor
    
    if draw_line:
        result = cv2.line(img, (int(xtop),int(ytop)), (int(xbot),int(ybot)), (0, 0, 255), 3)
    
    return result, angle

def compute_rocking(depth_img, colorized, skeleton, prev_z): 
    
    # extract key points 
    x_ls, y_ls = int(skeleton.joints[5][0]), int(skeleton.joints[5][1])
    x_rs, y_rs = int(skeleton.joints[2][0]), int(skeleton.joints[2][1])
    x_mid, y_mid = int(skeleton.joints[1][0]), int(skeleton.joints[1][1])
    z_ls, z_rs, z_mid = depth_img[y_ls, x_ls], depth_img[y_rs, x_rs], depth_img[y_mid, x_mid]
        
    # compare to last z location 
    avg_position = np.mean([z_ls, z_rs, z_mid])
    
    print('ROCKING: ', prev_z, avg_position)
    
    # display bounding box and status of rocking
    if abs(prev_z - avg_position) > .05: 
        cv2.putText(colorized, 'Rocking Movement: Bad', (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)
        cv2.rectangle(colorized, (x_rs, y_rs-20), (x_ls + 20, y_ls + 20), (255,12,36), 2)
        return 0
    else: 
        cv2.putText(colorized, 'Rocking Movement: Good', (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2)
        return 1
        
def computed_side_movement(img, skeleton, prev_x): 
    
    # extract key points 
    x_ls, y_ls = int(skeleton.joints[5][0]), int(skeleton.joints[5][1])
    x_rs, y_rs = int(skeleton.joints[2][0]), int(skeleton.joints[2][1])
    
    # compare to previous x value and display 
    avg_position = np.mean([x_ls, x_rs])
    
    print('SIDE MOVEMENT: ', prev_x, avg_position)
    
    if abs(prev_x - avg_position) > 5: 
        cv2.putText(img, 'Side Movement: Bad', (20, 65), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)
        cv2.rectangle(img, (x_rs, y_rs-20), (x_ls + 20, y_ls + 20), (255,12,36), 2)
        return 0
    else: 
        cv2.putText(img, 'Side Movement: Good', (20, 65), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2)
        return 1
    
def compute_shoulder_lifts(img, skeleton, ref_y): 
    
    # extract key points 
    y_rs, y_ls = int(skeleton.joints[2][1]), int(skeleton.joints[5][1])
    avg_pos = np.mean([y_rs, y_ls])
    
    print('SHOULDER LIFT: ', ref_y, avg_pos)
    
    if abs(ref_y - avg_pos) > 5: 
        cv2.putText(img, 'Shoulder Movement: Bad', (20, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)
        cv2.rectangle(img, (x_rs, y_rs-20), (x_ls + 20, y_ls + 20), (255,12,36), 2)
        return 0
    else: 
        cv2.putText(img, 'Shoulder Movement: Good', (20, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2)
        return 1
    
def compute_neck_movement(img, skeleton, ref_neck_pos):
    
    avg_pos = np.mean([int(skeleton.joints[0][0]), int(skeleton.joints[14][0]), int(skeleton.joints[15][0]), int(skeleton.joints[16][0]), int(skeleton.joints[17][0])])
    print('NECK MOVEMENT: ', ref_neck_pos, avg_pos)
    if abs(ref_neck_pos - avg_pos) > 10: 
        cv2.putText(img, 'Neck Movement: Bad', (20, 115), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)
        return 0 
    else: 
        cv2.putText(img, 'Neck Movement: Good', (20, 115), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2)
        return 1
    
# =====================================================
# COMPUTING LUNG PARAMETER FUNCTIONS  
# =====================================================     
    
def compute_keypoints(vals, display=False): 
    
    x = np.arange(len(vals))
    
    # find peaks
    peak_idxs = find_peaks(vals, distance=len(vals)//10)[0]
    peaks = np.array([vals[i] for i in peak_idxs])
    
    if len(peaks) < 3:
        print('not enough peaks computed, bad signal')
        return [-1] * 4

    # find minima
    minima_idxs = find_peaks(-vals, distance=len(vals)//10)[0]
    minimas = np.array([vals[i] for i in minima_idxs])
    
    # start of tidal 
    start = (0, vals[0])
    
    # end of exhale
    exhale_end_idx = peaks.argsort()[-1]
    exhale_end = (peak_idxs[exhale_end_idx], peaks[exhale_end_idx])
    
    # start of exhale 
    minima_coords = [(x, y) for x, y in zip(minima_idxs, minimas) if x < exhale_end[0]]
    minima_coords.sort(key=lambda x: x[1])
    exhale_start = minima_coords[0]
    
    # end of tidal
    peak_coords = [(x, y) for x, y in zip(peak_idxs, peaks) if x < exhale_end[0]]
    x_diffs = np.array([abs(x - exhale_end[0]) for x, _ in peak_coords])
    min_idx = x_diffs.argsort()[0]
    tidal_end = peak_coords[min_idx]
    
    if display: 
        
        plt.figure(1)
        plt.figure(figsize=(10,5))
        plt.plot(depth_chest_displacement, '-')
        plt.legend(['smoothed'], loc='best')

        # annotate effort points
        plt.annotate('start of tidal', start, 
                    xytext=(start[0] + 10, start[1] + .002), arrowprops=dict(arrowstyle="->"))
        plt.annotate('end of tidal', tidal_end,
                    xytext=(tidal_end[0] + 10, tidal_end[1] + .002), arrowprops=dict(arrowstyle="->"))
        plt.annotate('start of exhale', exhale_start,
                    xytext=(exhale_start[0] + 10, exhale_start[1] + .002), arrowprops=dict(arrowstyle="->"))
        plt.annotate('end of exhale', exhale_end,
                    xytext=(exhale_end[0] + 10, exhale_end[1] + .002), arrowprops=dict(arrowstyle="->"))

        plt.show()
    
    return start, tidal_end, exhale_start, exhale_end  

def compute_pft_measures(exhalation, flow_volume, flow_volume_exhalation):
    
    result = []
    
    # FVC = Volume After Blast - Volume After Inhalation
    FVC = exhalation.max() - 0
    print('FVC: ', round(FVC, 2)) 
    
    result.append(FVC)

    # FEV1 = Volume of air exhaled after one second (during exhalation blast)
    # signal is downsampled to 30 fps, so 30 frames = 1s 
    FEV1 = exhalation[6]
    print('FEV1: ', round(FEV1, 2))
    
    result.append(FEV1)

    # FEV1 / FVC 
    fev1_fvc_ratio = FEV1 / FVC
    print('FEV1 / FVC: ', round(fev1_fvc_ratio * 100, 2))
    
    result.append(fev1_fvc_ratio)

    # PEF = Maximum Flow 
    PEF = flow_volume.max()
    print('PEF: ', PEF)
    
    result.append(PEF)

    # FEF_25 = Flow of exhaled air at 25% FVC 
    fef_25_idxs = np.where(exhalation >= FVC * .25)[0]
    FEF_25 = flow_volume_exhalation[fef_25_idxs[1]]
    print('FEF_25: ', FEF_25)
    
    result.append(FEF_25)

    # FEF_50 = Flow of exhaled air at 50% FVC 
    fef_50_idxs = np.where(exhalation >= FVC * .50)[0]
    FEF_50 = flow_volume_exhalation[fef_50_idxs[1]]
    print('FEF_50: ', FEF_50)
    
    result.append(FEF_50)

    # FEF_75 = Flow of exhaled air at 75% FVC 
    fef_75_idxs = np.where(exhalation >= FVC * .75)[0]
    FEF_75 = flow_volume_exhalation[fef_75_idxs[1]]
    print('FEF_75: ', FEF_75)
    
    result.append(FEF_75)

    # FEF_25_75 = Mean forced exp flow between .25 and .75
    # (.75*FVC - .25*FVC) - (time(.25FVC) - time(.75FVC))
    FEF_25_75 = (.75*FVC - .25*FVC) / (abs(fef_25_idxs[1] - fef_75_idxs[1]))
    print('FEF_25_75: ', FEF_25_75)
    
    result.append(FEF_25_75)
    
    return result
                
# =====================================================
# 3D CHEST VOLUME POINTCLOUD FUNCTIONS  
# =====================================================       
        
# License: Apache 2.0. See LICENSE file in root directory.
# Copyright(c) 2015-2017 Intel Corporation. All Rights Reserved.

"""
OpenCV and Numpy Point cloud Software Renderer
This sample is mostly for demonstration and educational purposes.
It really doesn't offer the quality or performance that can be
achieved with hardware acceleration.
Usage:
------
Mouse: 
    Drag with left button to rotate around pivot (thick small axes), 
    with right button to translate and the wheel to zoom.
Keyboard: 
    [p]     Pause
    [r]     Reset View
    [d]     Cycle through decimation values
    [z]     Toggle point scaling
    [c]     Toggle color source
    [s]     Save PNG (./out.png)
    [e]     Export points to ply (./out.ply)
    [q\ESC] Quit
"""

class AppState:

    def __init__(self, *args, **kwargs):
        self.WIN_NAME = 'RealSense'
        self.pitch, self.yaw = math.radians(-10), math.radians(-15)
        self.translation = np.array([0, 0, -1], dtype=np.float32)
        self.distance = 2
        self.prev_mouse = 0, 0
        self.mouse_btns = [False, False, False]
        self.paused = False
        self.decimate = 1
        self.scale = True
        self.color = True
        self.lung_measure = False
        self.set_baseline = True
        self.compute_volume = False
        self.laser = True

    def reset(self):
        self.pitch, self.yaw, self.distance = 0, 0, 2
        self.translation[:] = 0, 0, -1

    @property
    def rotation(self):
        Rx, _ = cv2.Rodrigues((self.pitch, 0, 0))
        Ry, _ = cv2.Rodrigues((0, self.yaw, 0))
        return np.dot(Ry, Rx).astype(np.float32)

    @property
    def pivot(self):
        return self.translation + np.array((0, 0, self.distance), dtype=np.float32)


state = AppState()

def mouse_cb(event, x, y, flags, param):

    if event == cv2.EVENT_LBUTTONDOWN:
        state.mouse_btns[0] = True

    if event == cv2.EVENT_LBUTTONUP:
        state.mouse_btns[0] = False

    if event == cv2.EVENT_RBUTTONDOWN:
        state.mouse_btns[1] = True

    if event == cv2.EVENT_RBUTTONUP:
        state.mouse_btns[1] = False

    if event == cv2.EVENT_MBUTTONDOWN:
        state.mouse_btns[2] = True

    if event == cv2.EVENT_MBUTTONUP:
        state.mouse_btns[2] = False

    if event == cv2.EVENT_MOUSEMOVE:

        h, w = out.shape[:2]
        dx, dy = x - state.prev_mouse[0], y - state.prev_mouse[1]

        if state.mouse_btns[0]:
            state.yaw += float(dx) / w * 2
            state.pitch -= float(dy) / h * 2

        elif state.mouse_btns[1]:
            dp = np.array((dx / w, dy / h, 0), dtype=np.float32)
            state.translation -= np.dot(state.rotation, dp)

        elif state.mouse_btns[2]:
            dz = math.sqrt(dx**2 + dy**2) * math.copysign(0.01, -dy)
            state.translation[2] += dz
            state.distance -= dz

    if event == cv2.EVENT_MOUSEWHEEL:
        dz = math.copysign(0.1, flags)
        state.translation[2] += dz
        state.distance -= dz

    state.prev_mouse = (x, y)

def project(v):
    """project 3d vector array to 2d"""
    h, w = out.shape[:2]
    view_aspect = float(h)/w

    # ignore divide by zero for invalid depth
    with np.errstate(divide='ignore', invalid='ignore'):
        proj = v[:, :-1] / v[:, -1, np.newaxis] * \
            (w*view_aspect, h) + (w/2.0, h/2.0)

    # near clipping
    znear = 0.03
    proj[v[:, 2] < znear] = np.nan
    return proj


def view(v):
    """apply view transformation on vector array"""
    return np.dot(v - state.pivot, state.rotation) + state.pivot - state.translation


def line3d(out, pt1, pt2, color=(0x80, 0x80, 0x80), thickness=1):
    """draw a 3d line from pt1 to pt2"""
    p0 = project(pt1.reshape(-1, 3))[0]
    p1 = project(pt2.reshape(-1, 3))[0]
    if np.isnan(p0).any() or np.isnan(p1).any():
        return
    p0 = tuple(p0.astype(int))
    p1 = tuple(p1.astype(int))
    rect = (0, 0, out.shape[1], out.shape[0])
    inside, p0, p1 = cv2.clipLine(rect, p0, p1)
    if inside:
        cv2.line(out, p0, p1, color, thickness, cv2.LINE_AA)


def grid(out, pos, rotation=np.eye(3), size=1, n=10, color=(0x80, 0x80, 0x80)):
    """draw a grid on xz plane"""
    pos = np.array(pos)
    s = size / float(n)
    s2 = 0.5 * size
    for i in range(0, n+1):
        x = -s2 + i*s
        line3d(out, view(pos + np.dot((x, 0, -s2), rotation)),
               view(pos + np.dot((x, 0, s2), rotation)), color)
    for i in range(0, n+1):
        z = -s2 + i*s
        line3d(out, view(pos + np.dot((-s2, 0, z), rotation)),
               view(pos + np.dot((s2, 0, z), rotation)), color)


def axes(out, pos, rotation=np.eye(3), size=0.075, thickness=2):
    """draw 3d axes"""
    line3d(out, pos, pos +
           np.dot((0, 0, size), rotation), (0xff, 0, 0), thickness)
    line3d(out, pos, pos +
           np.dot((0, size, 0), rotation), (0, 0xff, 0), thickness)
    line3d(out, pos, pos +
           np.dot((size, 0, 0), rotation), (0, 0, 0xff), thickness)


def frustum(out, intrinsics, color=(0x40, 0x40, 0x40)):
    """draw camera's frustum"""
    orig = view([0, 0, 0])
    w, h = intrinsics.width, intrinsics.height

    for d in range(1, 6, 2):
        def get_point(x, y):
            p = rs.rs2_deproject_pixel_to_point(intrinsics, [x, y], d)
            line3d(out, orig, view(p), color)
            return p

        top_left = get_point(0, 0)
        top_right = get_point(w, 0)
        bottom_right = get_point(w, h)
        bottom_left = get_point(0, h)

        line3d(out, view(top_left), view(top_right), color)
        line3d(out, view(top_right), view(bottom_right), color)
        line3d(out, view(bottom_right), view(bottom_left), color)
        line3d(out, view(bottom_left), view(top_left), color)


def pointcloud(out, verts, texcoords, color, painter=True):
    """draw point cloud with optional painter's algorithm"""
    if painter:
        # Painter's algo, sort points from back to front

        # get reverse sorted indices by z (in view-space)
        # https://gist.github.com/stevenvo/e3dad127598842459b68
        v = view(verts)
        s = v[:, 2].argsort()[::-1]
        proj = project(v[s])
    else:
        proj = project(view(verts))

    if state.scale:
        proj *= 0.5**state.decimate

    h, w = out.shape[:2]

    # proj now contains 2d image coordinates
    j, i = proj.astype(np.uint32).T

    # create a mask to ignore out-of-bound indices
    im = (i >= 0) & (i < h)
    jm = (j >= 0) & (j < w)
    m = im & jm

    cw, ch = color.shape[:2][::-1]
    if painter:
        # sort texcoord with same indices as above
        # texcoords are [0..1] and relative to top-left pixel corner,
        # multiply by size and add 0.5 to center
        v, u = (texcoords[s] * (cw, ch) + 0.5).astype(np.uint32).T
    else:
        v, u = (texcoords * (cw, ch) + 0.5).astype(np.uint32).T
    # clip texcoords to image
    np.clip(u, 0, ch-1, out=u)
    np.clip(v, 0, cw-1, out=v)

    # perform uv-mapping
    out[i[m], j[m]] = color[u[m], v[m]]
    
def convert_depth_frame_to_pointcloud(depth_image, camera_intrinsics):
    """
    Convert the depthmap to a 3D point cloud

    Parameters:
    -----------
    depth_frame : rs.frame()
    The depth_frame containing the depth map
    camera_intrinsics : The intrinsic values of the imager in whose coordinate system the depth_frame is computed

    Return:
    ----------
    x : array
        The x values of the pointcloud in meters
    y : array
        The y values of the pointcloud in meters
    z : array
        The z values of the pointcloud in meters

    """

    [height, width] = depth_image.shape

    nx = np.linspace(0, width-1, width)
    ny = np.linspace(0, height-1, height)
    u, v = np.meshgrid(nx, ny)
    x = (u.flatten() - camera_intrinsics.ppx)/camera_intrinsics.fx
    y = (v.flatten() - camera_intrinsics.ppy)/camera_intrinsics.fy

    z = depth_image.flatten() / 1000;
    x = np.multiply(x,z)
    y = np.multiply(y,z)

    x = x[np.nonzero(z)]
    y = y[np.nonzero(z)]
    z = z[np.nonzero(z)]

    return x, y, z

def calculate_distance(p1, p2, depth_frame, intrinsics):
    
    ix,iy = p1
    x, y = p2
    
    print(p1, p2)
    
    udist = depth_frame.get_distance(ix, iy)
    vdist = depth_frame.get_distance(x, y)

    point1 = rs.rs2_deproject_pixel_to_point(intrinsics, [ix, iy], udist)
    point2 = rs.rs2_deproject_pixel_to_point(intrinsics, [x, y], vdist) 

    dist = math.sqrt(
        math.pow(point1[0] - point2[0], 2) + math.pow(point1[1] - point2[1],2) + math.pow(point1[2] - point2[2], 2))
    
    return dist

### Main (Starts Program)

In [4]:
#
# Configuration and Variable Setup
#

# configure depth and color streams of the intel realsense
config = rs.config()
config.enable_stream(rs.stream.depth, 1280, 720, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)

# start the realsense pipeline
pipeline = rs.pipeline()
cfg = pipeline.start(config)

device = cfg.get_device()
depth_sensor = device.query_sensors()[0]

# get intrinsics
profile = cfg.get_stream(rs.stream.depth) # Fetch stream profile for depth stream
camera_intrinsics = profile.as_video_stream_profile().get_intrinsics()

depth_profile = rs.video_stream_profile(profile)
depth_intrinsics = depth_profile.get_intrinsics()
w, h = depth_intrinsics.width, depth_intrinsics.height

depth_sensor = cfg.get_device().first_depth_sensor()
pixel_to_meters_scaling = depth_sensor.get_depth_scale()

# create necessary realsense objects
pc = rs.pointcloud()
decimate = rs.decimation_filter()
decimate.set_option(rs.option.filter_magnitude, 2 ** state.decimate)
colorizer = rs.colorizer()
align = rs.align(rs.stream.color)
filters = [rs.decimation_filter(),
           rs.disparity_transform(True),
           rs.spatial_filter(),
           rs.temporal_filter(), 
           rs.disparity_transform(False),
           rs.hole_filling_filter()]

# initialize the cubemos api (skeleton tracking)
skeletrack = skeletontracker(cloud_tracking_api_key="")
joint_confidence = 0.2

# create window for result
cv2.namedWindow(state.WIN_NAME, cv2.WINDOW_AUTOSIZE)
cv2.setMouseCallback(state.WIN_NAME, mouse_cb)

# initialize variables
skeleton_tracking_results = [] 
original_depths_imgs = []
depth_colorized_imgs = []
baseline_chest_positions = [] 
verts_list = []
texture_coords_list = []

chest_depth_rois = []
chest_color_rois = []
colorized_depths = []
chest_metrics = []

neck_movement_status_list = []
shoulder_movement_status_list = []
side_movement_status_list = []
rocking_movement_status_list = []
foot_pos_status_list = []

# save directory
demo_name = 'trial_10'
save_dir = 'D:/realsense_demos/at_home_1/' 

# load_model 
lg = joblib.load("D:/realsense_demos/models/latest_model.pkl")

save = False

if save: 
    os.makedirs(os.path.join(save_dir, demo_name, 'skeletons'), exist_ok=True)
    os.makedirs(os.path.join(save_dir, demo_name, 'depth_imgs'), exist_ok=True)
    os.makedirs(os.path.join(save_dir, demo_name, 'depth_color_imgs'), exist_ok=True)
    os.makedirs(os.path.join(save_dir, demo_name, 'verts'), exist_ok=True)
    os.makedirs(os.path.join(save_dir, demo_name, 'texture_coords'), exist_ok=True)

# initialize graph to display chest displacement
fig = plt.figure()

x1, y1 = [], [] 
line1, = plt.plot(x1, y1, 'k') 

# redraw the canvas
fig.canvas.draw()

# convert canvas to image
img = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8,
        sep='')
img  = img.reshape(fig.canvas.get_width_height()[::-1] + (3,))
img = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)

i = 0 
out = np.empty((h, w, 3), dtype=np.uint8)
alg_start = time.time()
while True:
    
    now = time.time()

    if state.laser: 
        depth_sensor.set_option(rs.option.emitter_enabled, 1)
    else: 
        depth_sensor.set_option(rs.option.emitter_enabled, 0)

    #
    # Collecting Necessary Data From RealSense
    #
    
    # Create a pipeline object.  
    # This object configures the streaming camera and owns it's handle
    unaligned_frames = pipeline.wait_for_frames()
    
    # obtain alligned depth and color images   
    frames = align.process(unaligned_frames)
    depth = frames.get_depth_frame()         
    color = frames.get_color_frame()
    depth_orig = depth
        
    # skip if not any are missing 
    if not depth or not color:
        continue
    
    # post process filtering for depth image 
    for f in filters:
        depth = f.process(depth)
    depth = depth.as_depth_frame()   
    depth_intrinsics = depth.profile.as_video_stream_profile().intrinsics
    w, h = depth_intrinsics.width, depth_intrinsics.height
    cv2.resizeWindow(state.WIN_NAME, w, h)
    #cv2.resizeWindow(window_name_2, w, h)
    
    # Convert frames to necessary numpy arrays 
    depth_image = np.asanyarray(depth.get_data())
    if state.lung_measure and save: 
        np.save(os.path.join(save_dir, demo_name, 'depth_imgs', 'depth_img_' + str(i) + '.npy'), depth_image)
    depth_image_color = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.1), cv2.COLORMAP_JET)
    depth_image_color_pc = depth_image_color.copy()
    if state.lung_measure and save: 
        np.save(os.path.join(save_dir, demo_name, 'depth_color_imgs', 'depth_colorized_' + str(i) + '.npy'), depth_image_color_pc)
    depth_scaled = depth_image * pixel_to_meters_scaling

    color_image = np.asanyarray(color.get_data())
    color_image = cv2.resize(color_image, dsize=(depth_image.shape[1], depth_image.shape[0]), interpolation=cv2.INTER_CUBIC)
    color_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)
    
    #
    # Run Skeleton Tracking & Movement Detections
    #
    
    # perform inference and update the tracking id
    skeletons = skeletrack.track_skeletons(color_image)
    
    if state.lung_measure and save: 
        np.save(os.path.join(save_dir, demo_name, 'skeletons', 'skeleton_coords_' + str(i) + '.npy'), skeletons[0])
        
    # place boxes for info 
    box_coords = ((5, 20), (5 + 255, 20 + 130))
    cv2.rectangle(depth_image_color, box_coords[0], box_coords[1], ((255, 255, 255)), cv2.FILLED)
    cv2.rectangle(depth_image_color, (450, 20), (645, 150), ((255, 255, 255)), cv2.FILLED)
    
    # render leg position on image 
    if len(skeletons) > 0: 
        try: 
            leg_position_status = detect_leg_pos_and_angle(depth_image_color, skeletons[0])
            if state.lung_measure and save: 
                foot_pos_status_list.append(leg_position_status)
        except:
            print('error in leg position and angle')
        try:    
            if state.set_baseline: 
                original_x = np.mean([skeletons[0].joints[5][0], skeletons[0].joints[2][0]])
            side_movement_status = computed_side_movement(depth_image_color, skeletons[0], original_x)
            if state.lung_measure and save: 
                side_movement_status_list.append(side_movement_status)
        except Exception as e: 
            print('error in side to side movement')
            print(e)
        try: 
            if state.set_baseline: 
                x_ls, y_ls = int(skeletons[0].joints[5][0]), int(skeletons[0].joints[5][1])
                x_rs, y_rs = int(skeletons[0].joints[2][0]), int(skeletons[0].joints[2][1])
                x_mid, y_mid = int(skeletons[0].joints[1][0]), int(skeletons[0].joints[1][1])
                z_ls, z_rs, z_mid = depth_scaled[y_ls, x_ls], depth_scaled[y_rs, x_rs], depth_scaled[y_mid, x_mid]
                ref_z = np.mean([z_ls, z_rs, z_mid])
            rocking_status = compute_rocking(depth_scaled, depth_image_color, skeletons[0], ref_z)
            if state.lung_measure and save: 
                rocking_movement_status_list.append(rocking_status)
        except Exception as e: 
            print('error in rocking')
            print(e)
        try:
            if state.set_baseline: 
                ref_shoulder_height = np.mean([y_rs, y_ls])
            shoulder_lift_status = compute_shoulder_lifts(depth_image_color, skeletons[0], ref_shoulder_height)
            if state.lung_measure and save: 
                shoulder_lift_status_list.append(shoulder_lift_status)
        except Exception as e: 
            print('error shoulder height')
            print(e)
        try:
            if state.set_baseline: 
                neck_features = [int(skeletons[0].joints[0][0]), int(skeletons[0].joints[14][0]), int(skeletons[0].joints[15][0]), int(skeletons[0].joints[16][0]), int(skeletons[0].joints[17][0])]
                ref_neck = np.mean(neck_features)
            neck_status = compute_neck_movement(depth_image_color, skeletons[0], ref_neck)
            if state.lung_measure and save: 
                neck_movement_status_list.append(neck_status)
        except Exception as e: 
            print('error neck movement')
            print(e)
    
    cm.render_result(skeletons, depth_image_color, joint_confidence)
    render_ids_3d(depth_image_color, skeletons, depth, depth_intrinsics, joint_confidence)
    
    #
    # Obtain Chest Region of Interests & 
    # Compute chest displacement and volume 
    #
    
    # obtain baseline region of interest for chest using skeleton points
    img = cv2.resize(img, dsize=(depth_image.shape[1], depth_image.shape[0]), interpolation=cv2.INTER_CUBIC)
    if state.set_baseline and len(skeletons) > 0:  
        curr_y_ls, curr_y_rs = int(skeletons[0].joints[5][1]), int(skeletons[0].joints[2][1])
        curr_x_ls = int(skeletons[0].joints[5][0])
        curr_x_rs = int(skeletons[0].joints[2][0])
        curr_x_la, curr_x_ra = int(skeletons[0].joints[6][0]), int(skeletons[0].joints[3][0])
        curr_x_lw, curr_y_lw = int(skeletons[0].joints[11][0]), int(skeletons[0].joints[11][1])
        curr_x_rw, curr_y_rw = int(skeletons[0].joints[8][0]), int(skeletons[0].joints[8][1])

        roi_y0 = max(curr_y_ls, curr_y_rs) # top height of roi 
        roi_y1 = min(curr_y_lw, curr_y_rw) # bottom height of roi 
        roi_y1 = int(roi_y0 + ((roi_y1 - roi_y0) * .7)) # bottom height of roi 
        roi_y0 -= 10
        
    baseline_chest_pos = round(np.mean(depth_scaled[roi_y0:roi_y1,curr_x_rw:curr_x_lw]), 4)
    cv2.rectangle(depth_image_color, (curr_x_rs, roi_y0), (curr_x_lw, roi_y1), (0,0,0), 2)
    chest_depth_rois.append(depth_scaled[roi_y0:roi_y1,curr_x_ra:curr_x_la])
    chest_color_rois.append(depth_image_color[roi_y0:roi_y1,curr_x_ra:curr_x_la])
    colorized_depths.append(depth_image_color)
    
    #print('CHEST DISPARITY: ', chest_displacements[-1])
    
    cv2.rectangle(depth_image_color, (5, 200), (5 + 255, 200 + 125), (255, 255, 255), cv2.FILLED)
    
    # compute sitting heights 
    chair_surface_pt = (int(np.mean([int(skeletons[0].joints[12][0]), int(skeletons[0].joints[9][0])])), int(np.mean([skeletons[0].joints[12][1], skeletons[0].joints[9][1]])))
    avg_head_x = np.mean([int(skeletons[0].joints[0][0]), int(skeletons[0].joints[14][0]), int(skeletons[0].joints[15][0]), int(skeletons[0].joints[16][0]), int(skeletons[0].joints[17][0])])
    avg_head_y = np.mean([int(skeletons[0].joints[0][1]), int(skeletons[0].joints[14][1]), int(skeletons[0].joints[15][1]), int(skeletons[0].joints[16][1]), int(skeletons[0].joints[17][1])])
    head_pt = (int(avg_head_x), int(avg_head_y)) 
    curr_x_ls, curr_y_ls = int(skeletons[0].joints[5][0]), int(skeletons[0].joints[5][1])
    curr_x_rs, curr_y_rs = int(skeletons[0].joints[2][0]), int(skeletons[0].joints[2][1])
    avg_shoulder_pt = (int(np.mean([int(skeletons[0].joints[5][0]), int(skeletons[0].joints[2][0])])), int(np.mean([int(skeletons[0].joints[5][1]), int(skeletons[0].joints[2][1])])))
    chair_surface_to_head = calculate_distance(chair_surface_pt, head_pt, depth_orig, camera_intrinsics)
    chair_surface_to_shoulder = calculate_distance(chair_surface_pt, avg_shoulder_pt, depth_orig, camera_intrinsics)
    
    print('CHAIR TO HEAD: ', chair_surface_to_head)
    print('CHAIR TO SHOULDER: ', chair_surface_to_shoulder)
    
    # compute chest and shoulder width measurements 
    curr_x_la, curr_x_ra = int(skeletons[0].joints[6][0]), int(skeletons[0].joints[3][0])
    chest_p1_y, chest_p2_y = int(np.mean([int(skeletons[0].joints[5][1]), int(skeletons[0].joints[11][1])])), int(np.mean([int(skeletons[0].joints[2][0]), int(skeletons[0].joints[8][1])]))
    
    chest_width_p1 = (curr_x_la, chest_p1_y)
    chest_width_p2 = (curr_x_ra, chest_p2_y)
    chest_width = calculate_distance(chest_width_p1, chest_width_p2, depth_orig, camera_intrinsics)
    shoulder_width = calculate_distance((curr_x_ls, curr_y_ls), (curr_x_rs, curr_y_rs), depth_orig, camera_intrinsics)
    
    # place text for lung function values (FVC, FEV1, FEV1/FVC)
    cv2.putText(depth_image_color, 'Chair-Head: ' + str(round(chair_surface_to_head, 3)), (465, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2)
    cv2.putText(depth_image_color, 'Chair-Should: ' + str(round(chair_surface_to_shoulder, 3)), (465, 65), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2)
    cv2.putText(depth_image_color, 'Chest-Width: ' + str(round(chest_width, 3)), (465, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2)
    cv2.putText(depth_image_color, 'Should-Width: ' + str(round(shoulder_width, 3)), (465, 115), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2)
    
    print('CHEST WIDTH: ', chest_width)
    print('SHOULDER WIDTH: ', shoulder_width)
    
    # place text for lung function values (FVC, FEV1, FEV1/FVC)
    cv2.putText(depth_image_color, 'FVC: ', (20, 225), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2)
    cv2.putText(depth_image_color, 'FEV1: ', (20, 250), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2)
    cv2.putText(depth_image_color, 'FEV1/FVC: ', (20, 275), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2)
    cv2.putText(depth_image_color, 'PEF: ', (20, 300), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2)
    
#     if not state.set_baseline: 
#         cv2.putText(depth_image_color, 'Compute Baseline: OFF', (20, 325), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2)
#     else:
#         cv2.putText(depth_image_color, 'Compute Baseline: ON', (20, 325), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2)
        
    #  
    # Realtime PointCloud Rendering
    #
    mapped_frame, color_source = depth, depth_image_color_pc
    pc.map_to(mapped_frame)
    points = pc.calculate(depth)
    
    #Pointcloud data to arrays
    verts, texcoords = np.asanyarray(points.get_vertices(2)), np.asanyarray(points.get_texture_coordinates(2))
    
    verts[np.where(verts[:,2]>baseline_chest_pos+.2)] = [np.nan, np.nan, np.nan]
    verts[np.where(verts[:,2]<baseline_chest_pos-.2)] = [np.nan, np.nan, np.nan]
    
    if state.lung_measure and save: 
        np.save(os.path.join(save_dir, demo_name, 'verts', 'vert_' + str(i) + '.npy'), verts)
        np.save(os.path.join(save_dir, demo_name, 'texture_coords', 'texcoords_' + str(i) + '.npy'), texcoords)
        
    # compute volume using pyntcloud or store distance
    if state.compute_volume:  
        
        # extract roi of chest 
        verts_copy = verts.reshape(h, w, 3)
        verts_copy = verts_copy[roi_y0:roi_y1,curr_x_ra:curr_x_la, :]
        verts_copy = verts_copy.reshape(verts_copy.shape[0] * verts_copy.shape[1], 3)
        
        df_points = pd.DataFrame(data=np.asanyarray(verts_copy), columns=['x', 'y', 'z'])
        df_points.dropna(inplace=True)
        
        try: 
            python_pcd = PyntCloud(points=df_points)
            convex_hull_id = python_pcd.add_structure("convex_hull")
            convex_hull = python_pcd.structures[convex_hull_id]
            chest_metrics.append(convex_hull.volume)
            cv2.putText(depth_image_color, 'Chest volume: ' + str(convex_hull.volume), (20, 300), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2)
        except: 
            continue
            
    else: 
        chest_metrics.append(np.mean(depth_scaled[roi_y0:roi_y1,curr_x_ra:curr_x_la]))
        #cv2.putText(depth_image_color, 'Chest  : ' + str(np.mean(depth_scaled[roi_y0:roi_y1,curr_x_ra:curr_x_la])), (20, 300), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2)
        cv2.putText(depth_image_color, 'CHEST: ' + str(baseline_chest_pos), (curr_x_rs, roi_y0 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2)
        
    if state.lung_measure: 
        x1.append(i)
        if state.compute_volume: 
            y1.append(-chest_metrics[-1]) 
        else: 
            y1.append(chest_metrics[-1])
        line1, = plt.plot(x1, y1, 'k') 

        # redraw the canvas
        fig.canvas.draw()

        # convert canvas to image
        img = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8,
                sep='')
        img  = img.reshape(fig.canvas.get_width_height()[::-1] + (3,))
        img = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
        img = cv2.resize(img, dsize=(depth_image.shape[1], depth_image.shape[0]), interpolation=cv2.INTER_CUBIC)
    
    # compute lung parameters
    if not state.lung_measure and len(y1) > 50: 
        
        # compute FVC of depth graph 
        depth_sensor_start, depth_sensor_tidal_end, depth_sensor_exhale_start, depth_sensor_exhale_end = compute_keypoints(np.array(y1))
        depthFVC = depth_sensor_exhale_end[1] - depth_sensor_exhale_start[1]

        # set info for model prediction 
        df_result = pd.DataFrame(columns=['DepthFVC', 'Height', 'Weight'])
        df_result.loc[len(df_result)] = [depthFVC, 70, 155]

        # predict 
        pred_results = lg.predict(df_result)
        predictedFVC = pred_results[0][0]

        # get depth graph from predicted FVC 
        final_test_volume = (y1 - y1[depth_sensor_exhale_start[0]]) * (predictedFVC / depthFVC)

        # compute depth sensor lung params 
        test_exhalation = final_test_volume[depth_sensor_exhale_start[0]:depth_sensor_exhale_end[0]]
        test_flow = np.gradient(final_test_volume, .1)
        test_flow_exhale = test_flow[depth_sensor_exhale_start[0]:depth_sensor_exhale_end[0]]
        test_pft_results = compute_pft_measures(test_exhalation, test_flow, test_flow_exhale)
        
        # display results
        cv2.putText(depth_image_color, 'FVC: ' + str(round(test_pft_results[0], 2)), (20, 225), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2)
        cv2.putText(depth_image_color, 'FEV1: ' + str(round(test_pft_results[1], 2)), (20, 250), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2)
        cv2.putText(depth_image_color, 'FEV1/FVC: ' + str(round(test_pft_results[2], 2)), (20, 275), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2)
        cv2.putText(depth_image_color, 'PEF: ' + str(round(test_pft_results[3], 2)), (20, 300), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2)

    # Render
    out.fill(0)

#     grid(out, (0, 0.5, 1), size=1, n=10)
#     frustum(out, depth_intrinsics)
#     axes(out, view([0, 0, 0]), state.rotation, size=0.1, thickness=1)

#     if not state.scale or out.shape[:2] == (h, w):
#         pointcloud(out, verts, texcoords, color_source)
#     else:
#         tmp = np.zeros((h, w, 3), dtype=np.uint8)
#         pointcloud(tmp, verts, texcoords, color_source)
#         tmp = cv2.resize(
#             tmp, out.shape[:2][::-1], interpolation=cv2.INTER_NEAREST)
#         np.putmask(out, tmp > 0, tmp)

#     if any(state.mouse_btns):
#         axes(out, view(state.pivot), state.rotation, thickness=4)

    dt = time.time() - now
    
    i += 1

    cv2.setWindowTitle(
        state.WIN_NAME, "RealSense (%dx%d) %dFPS (%.2fms) %s" %
        (w, h, 1.0/dt, dt*1000, "PAUSED" if state.paused else ""))
    
#     out_copy = out.copy()
#     out_copy = cv2.resize(out_copy, dsize=(depth_image_color.shape[1], depth_image_color.shape[0]), interpolation=cv2.INTER_CUBIC)
        
    #final = cv2.hconcat([depth_image_color, out_copy, img])
    final = cv2.hconcat([depth_image_color, img])
    cv2.imshow(state.WIN_NAME, final)
    key = cv2.waitKey(1)

    if key == ord("r"):
        state.reset()

    if key == ord("p"):
        state.paused ^= True

    if key == ord("d"):
        state.decimate = (state.decimate + 1) % 3
        decimate.set_option(rs.option.filter_magnitude, 2 ** state.decimate)

    if key == ord("z"):
        state.scale ^= True

    if key == ord("c"):
        state.color ^= True

    if key == ord("s"):
        cv2.imwrite('./out.png', out)

    if key == ord("e"):
        points.export_to_ply('./out.ply', mapped_frame)
        
    if key == 32:
        state.lung_measure = 1 - state.lung_measure
        
    if key == ord("b"):
        state.set_baseline = 1 - state.set_baseline
        
    if key == ord("v"):
        state.compute_volume = 1 - state.compute_volume
        
    if key == ord("l"):
        state.laser = 1 - state.laser
        
    if key == ord("e"):
        x1 = []
        y1 = []
        
        fig.clear()
        state.lung_measure = False

    if key in (27, ord("q")) or cv2.getWindowProperty(state.WIN_NAME, cv2.WND_PROP_AUTOSIZE) < 0:
        break
        
if save: 
    np.save(os.path.join(save_dir, demo_name, 'neck_movement_status_list.npy'), neck_movement_status_list)
    np.save(os.path.join(save_dir, demo_name, 'shoulder_movement_status_list.npy'), shoulder_movement_status_list)
    np.save(os.path.join(save_dir, demo_name, 'side_movement_status_list.npy'), side_movement_status_list)
    np.save(os.path.join(save_dir, demo_name, 'rocking_movement_status_list.npy'), rocking_movement_status_list)
    np.save(os.path.join(save_dir, demo_name, 'foot_pos_status_list.npy'), foot_pos_status_list)
    np.save(os.path.join(save_dir, demo_name, 'displacement.npy'), y1)

alg_end = time.time()
pipeline.stop()

print('ALG FPS: ', len(chest_depth_rois) / (alg_end - alg_start))

# # create window for lung volume graph
# flow_vol_name = 'Depth Camera - Lung Volume Graph'
# cv2.namedWindow(flow_vol_name, cv2.WINDOW_AUTOSIZE)

# # plot lung volume over time
# fig = plt.figure(2)
# length = len(final_test_volume)
# x_vals = np.linspace(0, 15, length)
# line1, = plt.plot(len, np.gradient(y1), 'k') 

# # redraw the canvas
# fig.canvas.draw()

# # convert canvas to image
# flow_img = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8,
#         sep='')
# flow_img  = img.reshape(fig.canvas.get_width_height()[::-1] + (3,))
# flow_img = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)

# cv2.imshow(flow_vol_name, flow_img)
# key = cv2.waitKey(0)

# cv2.destroyAllWindows()
# plt.show()

Initialising the Skeleton Tracking Pipeline with EDGE tracking.


The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


SIDE MOVEMENT:  371.3406677246094 371.0
ROCKING:  23.00633442607553 23.00633442607553
SHOULDER LIFT:  155.5 155.5
NECK MOVEMENT:  238.2 238.2
(386, 289) (238, 62)
(386, 289) (371, 155)
CHAIR TO HEAD:  0.0
CHAIR TO SHOULDER:  0.0
(440, 206) (310, 286)
(412, 164) (330, 147)
CHEST WIDTH:  0.8377297604819374
SHOULDER WIDTH:  0.0
SIDE MOVEMENT:  371.3406677246094 371.0
ROCKING:  1.7333334156622489 1.7333334156622489
SHOULDER LIFT:  155.5 155.5
NECK MOVEMENT:  238.2 238.2
(387, 289) (238, 62)
(387, 289) (371, 155)
CHAIR TO HEAD:  1.1958978208791289
CHAIR TO SHOULDER:  0.7607197742494981
(437, 206) (310, 288)
(412, 161) (330, 150)
CHEST WIDTH:  1.2186280921393071
SHOULDER WIDTH:  0.4343878521959132
SIDE MOVEMENT:  371.3406677246094 371.0
ROCKING:  1.7196667483464505 1.7196667483464505
SHOULDER LIFT:  155.5 155.5
NECK MOVEMENT:  238.2 238.2
(387, 291) (238, 62)
(387, 291) (371, 155)
CHAIR TO HEAD:  1.338480028769752
CHAIR TO SHOULDER:  0.7111781919199551
(437, 206) (310, 288)
(412, 164) (330, 

In [33]:
 1 / dt 

19.660830720048

In [ ]:
# create window for lung volume graph
flow_vol_name = 'Depth Camera - Lung Volume Graph'
cv2.namedWindow(flow_vol_name, cv2.WINDOW_AUTOSIZE)

# plot lung volume over time
fig = plt.figure(2)
length = len(final_test_volume)
x_vals = np.linspace(0, 20, length)
line1, = plt.plot(x_vals, final_test_volume, 'k') 

# redraw the canvas
fig.canvas.draw()

# convert canvas to image
vol_img = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8,
        sep='')
vol_img  = vol_img.reshape(fig.canvas.get_width_height()[::-1] + (3,))
vol_img = cv2.cvtColor(vol_img, cv2.COLOR_RGB2BGR)

# display figure 
cv2.imshow(flow_vol_name, vol_img)

key = cv2.waitKey(0)
cv2.destroyAllWindows()
plt.show()

The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


#### Compute Lung Params

In [12]:
def compute_keypoints(vals, display=False): 
    
    x = np.arange(len(vals))
    
    # find peaks
    peak_idxs = find_peaks(vals, distance=len(vals)//10)[0]
    peaks = np.array([vals[i] for i in peak_idxs])
    
    if len(peaks) < 3:
        print('not enough peaks computed, bad signal')
        return [-1] * 4

    # find minima
    minima_idxs = find_peaks(-vals, distance=len(vals)//10)[0]
    minimas = np.array([vals[i] for i in minima_idxs])
    
    # start of tidal 
    start = (0, vals[0])
    
    # end of exhale
    exhale_end_idx = peaks.argsort()[-1]
    exhale_end = (peak_idxs[exhale_end_idx], peaks[exhale_end_idx])
    
    # start of exhale 
    minima_coords = [(x, y) for x, y in zip(minima_idxs, minimas) if x < exhale_end[0]]
    minima_coords.sort(key=lambda x: x[1])
    exhale_start = minima_coords[0]
    
    # end of tidal
    peak_coords = [(x, y) for x, y in zip(peak_idxs, peaks) if x < exhale_end[0]]
    x_diffs = np.array([abs(x - exhale_end[0]) for x, _ in peak_coords])
    min_idx = x_diffs.argsort()[0]
    tidal_end = peak_coords[min_idx]
    
    if display: 
        
        plt.figure(1)
        plt.figure(figsize=(10,5))
        plt.plot(depth_chest_displacement, '-')
        plt.legend(['smoothed'], loc='best')

        # annotate effort points
        plt.annotate('start of tidal', start, 
                    xytext=(start[0] + 10, start[1] + .002), arrowprops=dict(arrowstyle="->"))
        plt.annotate('end of tidal', tidal_end,
                    xytext=(tidal_end[0] + 10, tidal_end[1] + .002), arrowprops=dict(arrowstyle="->"))
        plt.annotate('start of exhale', exhale_start,
                    xytext=(exhale_start[0] + 10, exhale_start[1] + .002), arrowprops=dict(arrowstyle="->"))
        plt.annotate('end of exhale', exhale_end,
                    xytext=(exhale_end[0] + 10, exhale_end[1] + .002), arrowprops=dict(arrowstyle="->"))

        plt.show()
    
    return start, tidal_end, exhale_start, exhale_end

def compute_pft_measures(exhalation, flow_volume, flow_volume_exhalation):
    
    result = []
    
    # FVC = Volume After Blast - Volume After Inhalation
    FVC = exhalation.max() - 0
    print('FVC: ', round(FVC, 2)) 
    
    result.append(FVC)

    # FEV1 = Volume of air exhaled after one second (during exhalation blast)
    # signal is downsampled to 30 fps, so 30 frames = 1s 
    FEV1 = exhalation[9]
    print('FEV1: ', round(FEV1, 2))
    
    result.append(FEV1)

    # FEV1 / FVC 
    fev1_fvc_ratio = FEV1 / FVC
    print('FEV1 / FVC: ', round(fev1_fvc_ratio * 100, 2))
    
    result.append(fev1_fvc_ratio)

    # PEF = Maximum Flow 
    PEF = flow_volume.max()
    print('PEF: ', PEF)
    
    result.append(PEF)

    # FEF_25 = Flow of exhaled air at 25% FVC 
    fef_25_idxs = np.where(exhalation >= FVC * .25)[0]
    FEF_25 = flow_volume_exhalation[fef_25_idxs[1]]
    print('FEF_25: ', FEF_25)
    
    result.append(FEF_25)

    # FEF_50 = Flow of exhaled air at 50% FVC 
    fef_50_idxs = np.where(exhalation >= FVC * .50)[0]
    FEF_50 = flow_volume_exhalation[fef_50_idxs[1]]
    print('FEF_50: ', FEF_50)
    
    result.append(FEF_50)

    # FEF_75 = Flow of exhaled air at 75% FVC 
    fef_75_idxs = np.where(exhalation >= FVC * .75)[0]
    FEF_75 = flow_volume_exhalation[fef_75_idxs[1]]
    print('FEF_75: ', FEF_75)
    
    result.append(FEF_75)

    # FEF_25_75 = Mean forced exp flow between .25 and .75
    # (.75*FVC - .25*FVC) - (time(.25FVC) - time(.75FVC))
    FEF_25_75 = (.75*FVC - .25*FVC) / (abs(fef_25_idxs[1] - fef_75_idxs[1]))
    print('FEF_25_75: ', FEF_25_75)
    
    result.append(FEF_25_75)
    
    return result

In [33]:
import joblib
from scipy.signal import find_peaks

In [34]:
# load_model 
lg = joblib.load("D:/realsense_demos/models/latest_model.pkl")

In [35]:
# compute FVC of depth graph 
depth_sensor_start, depth_sensor_tidal_end, depth_sensor_exhale_start, depth_sensor_exhale_end = compute_keypoints(np.array(y1))
depthFVC = depth_sensor_exhale_end[1] - depth_sensor_exhale_start[1]

# set info for model prediction 
df_result = pd.DataFrame(columns=['DepthFVC', 'Height', 'Weight'])
df_result.loc[len(df_result)] = [depthFVC, 70, 155]

# predict 
pred_results = lg.predict(df_result)
predictedFVC = pred_results[0][0]

# get depth graph from predicted FVC 
final_test_volume = (y1 - y1[depth_sensor_exhale_start[0]]) * (predictedFVC / depthFVC)

# compute depth sensor lung params 
test_exhalation = final_test_volume[depth_sensor_exhale_start[0]:depth_sensor_exhale_end[0]]
test_flow = np.gradient(final_test_volume, .1)
test_flow_exhale = test_flow[depth_sensor_exhale_start[0]:depth_sensor_exhale_end[0]]
test_pft_results = compute_pft_measures(test_exhalation, test_flow, test_flow_exhale)

FVC:  4.56
FEV1:  3.61
FEV1 / FVC:  79.2
PEF:  8.467649357410082
FEF_25:  6.932972943179696
FEF_50:  5.266968076181726
FEF_75:  3.131306576299129
FEF_25_75:  0.5704439118651352


In [36]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
axes[0].plot(final_test_volume)
axes[1].plot(test_flow)
fig.tight_layout()

In [12]:
len(y1/6)

TypeError: unsupported operand type(s) for /: 'list' and 'int'

In [26]:
len(y1) / 22

4.454545454545454

In [24]:
len(test_exhalation)

15

In [31]:
len(y1)

98